In [211]:
from utils.mapping import production_mapping
from utils.calculation import get_initial_data, get_data 
import pandas as pd
import numpy as np

In [213]:
# filtered_df, mapping_name, stocks_in_name = get_data(raw,'WCESTUS1')
# chart_df = chart_seasonality(filtered_df, mapping_name, stocks_in_name, True, True, True, True)
# chart_df.tail()

In [291]:
def extend_df(df):
    df = df.copy()
    extended_dates = pd.date_range(start=df['period'].iloc[-1] + pd.Timedelta(days=7), end='2024-12-27', freq='7D')
    extended_df = pd.DataFrame({'period': extended_dates})
    df = pd.concat([df, extended_df], ignore_index=True)
    return df

def add_weekOfYear_Year(df):
    # Insert week of year and year
    df = df.copy()
    df.insert(0,'week_of_year',df['period'].dt.isocalendar().week)
    df.insert(1,'year',df['period'].dt.year)
    df = df[df['week_of_year'] <= 52]
    return df

def select_seasonality_range(df, range_selector):
    df = df.copy()
    
    if range_selector == '1519':
        years = [2015, 2016, 2017, 2018, 2019]
    elif range_selector == '1823':
        years = [2018, 2019, 2021, 2022, 2023]
    
    def get_min_max(df, min_or_max, years):
        df = df.copy()
        df = df[df['year'].isin(years)]
        df = df.drop(columns=['period', 'year'])
        if min_or_max == 'min':
            df = df.groupby(['week_of_year']).min().reset_index()
        elif min_or_max == 'max':
            df = df.groupby(['week_of_year']).max().reset_index()
        return df

    def get_average(df, years):
        df = df[df['year'].isin(years)]
        df = df.drop(columns=['period', 'year'])
        df = df.groupby(['week_of_year']).mean().reset_index()
        df.insert(0, 'type', f'average_{range_selector}')    
        return df

    df_min = get_min_max(df, 'min', years)
    df_min.insert(0, 'type', f'min_{range_selector}')
    
    df_max = get_min_max(df, 'max', years)
    df_max.insert(0, 'type', f'max_{range_selector}')
    
    df_range = df_max.copy()
    for col in df_min.columns:
        if col not in ['type', 'week_of_year']:
            df_range[col] = df_max[col] - df_min[col]
    df_range['type'] = f'range_{range_selector}'
    
    df_average = get_average(df, years)
    
    dff = pd.concat([df_range, df_min, df_max,df_average])
    
    return dff


def get_seasonality_data(df):
    df = df.copy()
    # df = extend_df(df)
    df = add_weekOfYear_Year(df)

    df_1519 = select_seasonality_range(df, '1519')
    df_1823 = select_seasonality_range(df, '1823')    

    df.insert(0, 'type', 'actual')
    df['type'] = df['type'] + '_' + df['year'].astype(str)
    df.drop(columns=['year'], inplace=True)

    df = pd.concat([df,df_1519, df_1823])    
    df['week_of_year'] = df['week_of_year'].astype(int)
    
    df['period'] = pd.to_datetime(df['period'])
    
    
    def format_date(date):
        return date.strftime('%b %d') if pd.notnull(date) else 'No Date'    
    df['period'] = df['period'].apply(format_date)
    
    def format_value_case_insensitive(val, column_name):
        if "stocks" in column_name.lower():
            #divide by 1000
            val = val/1000
            return f"{val:,.1f}" if val >= 0 else f"({abs(val):,.1f})"
        else:
            return f"{val:,.0f}" if val >= 0 else f"({abs(val):,.0f})"

    df = df.set_index(['type','week_of_year','period'])
    cols = df.columns
    cols = [production_mapping[col] for col in cols]
    df.columns = cols

    for column in df.columns:
        df[column] = df[column].apply(lambda x: format_value_case_insensitive(x, column))

    reversed_mapping = {v: k for k, v in production_mapping.items()}
    df.columns = [reversed_mapping[col] for col in df.columns]
    
    #reset index
    df = df.reset_index()
    

    return df

raw = get_seasonality_data(get_initial_data())

In [361]:
df = raw.copy()
ids = list(production_mapping.keys())
ids = ids[:50]



def pivot_individual_data(df,id):
    
    def get_individual_data(df, id):
        cols = ['type','week_of_year','period', id] 
        df = df[cols]
        df = df.rename(columns={id: 'value'})
        return df    
    
    df = df.copy()
    df = get_individual_data(df, id)
    pivot_values = df.pivot_table(index=['week_of_year'], columns='type', values='value', aggfunc='first')
    pivot_dates = df[df['type'].str.contains('actual')].pivot_table(index=['week_of_year'], columns='type', values='period', aggfunc='first')
    pivot_dates.columns = [f'dates_{col}' for col in pivot_dates.columns]
    df = pd.concat([pivot_dates, pivot_values], axis=1)
    return df

df = pivot_individual_data(df, id)

dfs = {}
for id in ids:
    df = pivot_individual_data(raw, id)
    dfs = {**dfs, **{id: df}}


In [367]:
dfs['WCESTUS1']

,dates_actual_2015,dates_actual_2016,dates_actual_2017,dates_actual_2018,dates_actual_2019,dates_actual_2020,dates_actual_2021,dates_actual_2022,dates_actual_2023,dates_actual_2024,...,actual_2023,actual_2024,average_1519,average_1823,max_1519,max_1823,min_1519,min_1823,range_1519,range_1823
week_of_year,,,,,,,,,,,,,,,,,,,,,
1,Jan 02,Jan 08,Jan 06,Jan 05,Jan 04,Jan 03,Jan 08,Jan 07,Jan 06,Jan 05,...,439.6,432.4,428.5,438.9,483.1,482.2,348.8,413.3,134.3,68.9
2,Jan 09,Jan 15,Jan 13,Jan 12,Jan 11,Jan 10,Jan 15,Jan 14,Jan 13,Jan 12,...,448.0,429.9,428.9,439.6,485.5,486.6,354.2,412.7,131.3,73.9
3,Jan 16,Jan 22,Jan 20,Jan 19,Jan 18,Jan 17,Jan 22,Jan 21,Jan 20,Jan 19,...,448.5,420.7,434.5,439.6,488.3,476.7,364.3,411.6,124.0,65.1
4,Jan 23,Jan 29,Jan 27,Jan 26,Jan 25,Jan 24,Jan 29,Jan 28,Jan 27,Jan 26,...,452.7,421.9,440.7,441.6,494.8,475.7,373.1,415.1,121.6,60.5
5,Jan 30,Feb 05,Feb 03,Feb 02,Feb 01,Jan 31,Feb 05,Feb 04,Feb 03,Feb 02,...,455.1,427.4,445.2,440.4,508.6,469.0,379.5,410.4,129.1,58.6
6,Feb 06,Feb 12,Feb 10,Feb 09,Feb 08,Feb 07,Feb 12,Feb 11,Feb 10,Feb 09,...,471.4,439.4,449.5,443.5,518.1,471.4,383.8,411.5,134.3,59.9
7,Feb 13,Feb 19,Feb 17,Feb 16,Feb 15,Feb 14,Feb 19,Feb 18,Feb 17,Feb 16,...,479.0,443.0,452.3,446.6,518.7,479.0,391.5,416.0,127.2,63.0
8,Feb 20,Feb 26,Feb 24,Feb 23,Feb 22,Feb 21,Feb 26,Feb 25,Feb 24,Feb 23,...,480.2,447.2,455.2,449.5,520.2,484.6,399.9,413.4,120.2,71.2
9,Feb 27,Mar 04,Mar 03,Mar 02,Mar 01,Feb 28,Mar 05,Mar 04,Mar 03,Mar 01,...,478.5,448.5,461.7,453.5,528.4,498.4,410.2,411.6,118.1,86.8


In [ ]:
tickvals = [0, 5, 9, 14, 18, 22, 27, 31, 36, 40, 45, 49]
ticktext = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

def chart_seasonality(df, mapping_name, stocks_in_name, toggle_seag_range, toggle_2022,toggle_2023,toggle_2024):
    
    cols = ['type','week_of_year','period', id] 
    df = df[cols]

    min_1519_mask = df['type'] == 'min_1519'
    max_1519_mask = df['type'] == 'max_1519'
    average_1519_mask = df['type'] == 'average_1519'
    min_1823_mask = df['type'] == 'min_1823'
    average_1823_mask = df['type'] == 'average_1823'

    actual_2024_mask = df['type'] == 'actual_2024'
    actual_2023_mask = df['type'] == 'actual_2023'
    actual_2022_mask = df['type'] == 'actual_2022'    

    if not toggle_seag_range:
        min_rng = df[df['type']=='min_1519']
        max_rng = df[df['type']=='max_1519']
        avg_rng = df[df['type']=='average_1519']
    else:
        min_rng = df[df['type']=='min_1823']
        max_rng = df[df['type']=='min_1823']
        avg_rng = df[df['type']=='average_1823']

    period_2024 = df[df['type']=='actual_2024']
    period_2023 = df[df['type']=='actual_2023']
    period_2022 = df[df['type']=='actual_2022']

    traces = []


    traces.append(go.Scatter(
        x=min_rng['week_of_year'],
        y=min_rng['value'],
        mode='lines',
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ))

    traces.append(go.Scatter(
        x=max_rng['week_of_year'],
        y=max_rng['value'],
        mode='lines',
        fill='tonexty',
        fillcolor='#f4f5f9',
        line_color='#f4f5f9', 
        showlegend=False,
        hoverinfo='skip'
    ))

    traces.append(go.Scatter(
        x=avg_rng['week_of_year'],
        y=avg_rng['value'],
        mode='lines',
        line=dict(color='black', dash='dot', width=1),
        showlegend=False,
        hoverinfo='skip'
    ))

    if not toggle_2024:
        traces.append(go.Scatter(
            x=df['week_of_year'],
            y=df['value_2024'],
            mode='lines',
            name='2024',
            hoverinfo='text',
            text=df.apply(lambda row: f"{format_date(row['period_2024'])}: {format_value(row['value_2024'],stocks_in_name)}", axis=1),
            line=dict(color='#c00000', dash='solid', width=2),
        ))

    if not toggle_2023:
        traces.append(go.Scatter(
            x=df['week_of_year'],
            y=df['value_2023'],
            mode='lines',
            name='2023',
            hoverinfo='text',
            line=dict(color='#e97132', dash='solid', width=2),
            text=df.apply(lambda row: f"{format_date(row['period_2023'])}: {format_value(row['value_2023'],stocks_in_name)}", axis=1),
        ))

    if not toggle_2022:
        traces.append(go.Scatter(
            x=df['week_of_year'],
            y=df['value_2022'],
            mode='lines',
            name='2022',
            hoverinfo='text',
            line=dict(color='#bfbec4', dash='solid', width=2),
            text=df.apply(lambda row: f"{format_date(row['period_2022'])}: {format_value(row['value_2022'],stocks_in_name)}", axis=1),
        ))


    layout = go.Layout(

        title=dict(
            text=mapping_name,
            y=0.97,
            x=0.03,
            xanchor='left',
            yanchor='top',
            font=dict(
                color='black'
            )
        ),   

        xaxis=dict(
            range=[0, len(df['week_of_year']) - 1],
            tickmode='array',
            tickvals=tickvals,
            ticktext=ticktext,
            type='category',
            showspikes=True,
            spikedash='solid',
            spikecolor='grey',
            spikethickness=0.5,
            spikemode='across',
            showline=True,
            showgrid=False,
            linecolor='black',
            linewidth=0.5,
            zeroline=False
        ),
        yaxis=dict(
            tickformat=',.0fK',
            showgrid=False,
            showline=True,
            linecolor='black',
            linewidth=0.5,
            zeroline=False
        ),
        hovermode='x unified',
        template='plotly_white',
        legend=dict(
            x=0.85,
            y=1.0,
            xanchor='left',
            yanchor='top',
            orientation='v'
        ),
        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="Open Sans",
            bordercolor="#ccc"
        ),
        margin=dict(l=40, r=40, t=40, b=25),        
        showlegend=True
    )


    # Return the figure as a dictionary
    return {'data': traces, 'layout': layout}

